In [ ]:
from pynq import Overlay, MMIO
from time import sleep
from datetime import datetime

class NEC_IR_Controller:
    def __init__(self, bitfile_path):
        # Load the overlay
        self.ol = Overlay(bitfile_path)
        self.ol.download()
        self.list_ip = []
        self.base_ip = 0x4121_0000
        for i in range(10):
            self.new_ip = self.base_ip + hex(i)
            self.list_ip.append(self.new_ip)
        
        # Configure GPIO addresses (adjust these to match your Vivado design)
        self.AXI_GPIO = []
        for i in range(10):
            self.AXI_GPIO.append(self.list_ip[i])
        self.send_ctrl = 0x412B_0000
        # Initialize MMIO interfaces
        for i in range(10):
            self.AXI_GPIO[i] = MMIO(self.NEC_GPIO_BASE, 0x10000)
        self.send_ctrl = MMIO(self.NEC_GPIO_BASE, 0x10000)
        # Configure directions:
        for i in range(10):
            self.AXI_GPIO[i].write(0x04, 0x00000000)  # All outputs for NEC
            self.AXI_GPIO[i].write(0x0C, 0x00000000)  # All outputs for send control
        self.send_ctrl.write(0x04, 0x00000000)   
               
        # Timing parameters
        self.NEC_HOLD_TIME = 0.02  # 20ms hold for NEC frame
        self.BTN_PULSE_WIDTH = 0.001  # 1ms pulse width for button

    def send_own_command(self, addr,cmd):
        """Send a 8-bit OWN command to the PL transmitter with button trigger"""
        #clear everything
        for i in range(10):
            self.AXI_GPIO[i].write(0x00,0)
            self.AXI_GPIO[i].write(0x08,0)
        sleep(self.BTN_PULSE_WIDTH)
        # Step 1: Write the NEC command
        for i in range(10):
            self.AXI_GPIO[i].write(0x00,addr[i])
            self.AXI_GPIO[i].write(0x08,cmd[i])
        
        # Step 2: Pulse the button signal
        self.send_ctrl.write(0x00, 0x1)  # Assert button
        sleep(self.BTN_PULSE_WIDTH)
        self.send_ctrl.write(0x00, 0x0)  # Deassert button
        
        # Step 3: Hold the NEC command for the required duration
        sleep(self.NEC_HOLD_TIME)
        
        # Step 4: Clear the NEC command
        for i in range(10):
            self.AXI_GPIO[i].write(0x00,0)
            self.AXI_GPIO[i].write(0x08,0)
# Example usage:
if __name__ == "__main__":
    # Initialize with correct class name
    ir = NEC_IR_Controller(
        "/home/xilinx/jupyter_notebooks/xilinx/overlays/own/design_1_wrapper.bit"
    )
    
    try:
        a = datetime.now()
        addr = [0x0000,0x0001,0x0002,0x0003,0x0004,0x0005,0x0006,0x0007,0x0008,0x0009]
        cmd =  [0x0000,0x0001,0x0002,0x0003,0x0004,0x0005,0x0006,0x0007,0x0008,0x0009]
        ir.send_own_command(addr,cmd)
        print(datetime.now() - a)#estimating the time of i code sending
            
        
    except Exception as e:
        print(f"Error: {e}")
    finally:
        # Clean up GPIO states
        ir.nec_gpio.write(0x00, 0)
        ir.nec_gpio.write(0x08, 0)
        print("Cleanup complete")

0:00:25.531279
Cleanup complete
